# Image Warping and Mosaicing - Project 4 Part A

In [ ]:
# Import necessary modules
import os
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import skimage as sk
import skimage.io as skio
import cv2

## Part 1 - Shoot the Pictures

In [ ]:
# Load and display pictures I took
vlsb_left = sk.img_as_float(skio.imread("vlsb_left.jpg"))
vlsb_middle = sk.img_as_float(skio.imread("vlsb_middle.jpg"))
fig,axes = plt.subplots(1,2)
axes[0].imshow(vlsb_left)
axes[1].imshow(vlsb_middle)

In [ ]:
vlsb_left_pts = np.array([[161, 459],
                         [174, 579],
                         [205, 457],
                         [215, 579],
                         [210, 520],
                         [165, 520]])
vlsb_middle_pts = np.array([[126, 375],
                            [123, 506],
                            [168, 373],
                            [167, 508],
                            [167, 440],
                            [123, 440]])

## Part 2 - Recover Homographies

In [ ]:
def computeH(im1_pts, im2_pts):
    # Convert (kinda) input vectors (assumed to be column vectors)
    num_points = im1_pts.shape[0]
    A = np.zeros((num_points*2,8))
    for i in range(num_points):
        x1,y1 = im1_pts[i]
        x2,y2 = im2_pts[i]
        A[2*i] = [x1,y1,1,0,0,0,-x1*x2,-y1*x2]
        A[2*i+1] = [0,0,0,x1,y1,1,-x1*y2,-y1*y2]
    b = np.ravel(im2_pts, order = 'C')
    h = np.linalg.lstsq(A, b, rcond=None)[0]
    h = np.append(h,[1])
    return np.reshape(h, (3,3))

## Part 3 - Warp Images

In [ ]:
def getMaxSize(size, H):
    height, width, _ = size
    corners = [(0,0,1),(height-1,0,1),(0,width-1,1),(height-1,width-1,1)]
    x_max, y_max = -1,-1
    for c in corners:
        new_c = H @ c
        x_max = int(max(x_max, np.ceil(new_c[0]/new_c[2])))
        y_max = int(max(y_max, np.ceil(new_c[1]/new_c[2])))
    return (x_max+1,y_max+1)     

In [ ]:
def warpImage(im, H):
    max_size = getMaxSize(im.shape, H)
    # Warp image
    H_inv = np.linalg.inv(H)
    src_pts = np.array(list(np.ndindex(im.shape[:-1])))
    dst_pts = np.array(list(np.ndindex(max_size)))
    # Transform dst_point set
    transform_pts = (H_inv @ np.vstack([dst_pts.T,np.ones((1,dst_pts.shape[0]))])).T
    transform_pts = transform_pts[:,:2] / transform_pts[:,2].reshape(-1,1)
    values = np.array(list(zip(im[:,:,0].flatten(),im[:,:,1].flatten(),im[:,:,2].flatten())))
    #values = np.array(list(zip(im[:,:,0].flatten(),im[:,:,1].flatten(),im[:,:,2].flatten(),im[:,:,3].flatten())))
    # Interpolate
    interp = sp.interpolate.griddata(src_pts, values, transform_pts, method='linear', fill_value = 0)
    warped_im = np.reshape(interp,(max_size[0],max_size[1],3))
    return warped_im

In [ ]:
vlsb_left_H = computeH(vlsb_left_pts, vlsb_middle_pts)
warped_vlsb_left = warpImage(vlsb_left, vlsb_left_H)
plt.imshow(warped_vlsb_left)

In [ ]:
skio.imsave("warped_vlsb_left.jpg", sk.img_as_ubyte(warped_vlsb_left))

## Part 4 - Image Rectification

In [ ]:
# Load Book image
book = sk.img_as_float(skio.imread("book.jpg"))
plt.imshow(book)

In [ ]:
book_down_pts = np.array([[ 156,  481],
 [ 146,  886],
 [ 619,  316],
 [ 619, 1136]])


book_front_pts = np.array([[92,  452],
 [92,1116],
 [830,452],
 [830, 1116]])

In [ ]:
plt.imshow(book)
plt.plot(book_down_pts[:,1],book_down_pts[:,0],'ro')

In [ ]:
plt.imshow(book)
plt.plot(book_front_pts[:,1],book_front_pts[:,0],'ro')

In [ ]:
book_H_mat = computeH(book_down_pts,book_front_pts)
warped_book = warpImage(book, book_H_mat)
plt.imshow(warped_book)

In [ ]:
skio.imsave("book_front.jpg",sk.img_as_ubyte(warped_book))

In [ ]:
book_cropped = warped_book[book_front_pts[0,0]:book_front_pts[3,0],book_front_pts[0,1]:book_front_pts[3,1],:]
plt.imshow(book_cropped)
skio.imsave("book_front_cropped.jpg", sk.img_as_ubyte(book_cropped))

In [ ]:
# Load painting image
painting = sk.img_as_float(skio.imread("painting.jpg"))
plt.imshow(painting)

In [ ]:
painting_right_pts = np.array([[308, 639],
 [ 43, 958],
 [668, 625],
 [797, 978]])

painting_front_pts = np.array([[325,  456],
 [325, 1136],
 [793, 456],
 [ 793, 1136]])

In [ ]:
plt.imshow(painting)
plt.plot(painting_right_pts[:,1],painting_right_pts[:,0],'ro')

In [ ]:
plt.imshow(painting)
plt.plot(painting_front_pts[:,1],painting_front_pts[:,0],'ro')

In [ ]:
painting_H_mat = computeH(painting_right_pts,painting_front_pts)
warped_painting = warpImage(painting, painting_H_mat)
plt.imshow(warped_painting)

In [ ]:
skio.imsave("painting_front.jpg",sk.img_as_ubyte(warped_painting))

In [ ]:
painting_cropped = warped_painting[painting_front_pts[0,0]:painting_front_pts[3,0],painting_front_pts[0,1]:painting_front_pts[3,1],:]
plt.imshow(painting_cropped)
skio.imsave("painting_front_cropped.jpg", sk.img_as_ubyte(painting_cropped))

## Part 5 - Mosaic

In [ ]:
# Laplacian blending functions from project 2
def gaussian_stack(image):
    g_stack = [image]
    for depth in range(2):
        G = cv2.getGaussianKernel(50, 2**depth)
        gaussian_kernel = G @ G.T
        channels = []
        for i in range(3):
            channel = image[:,:,i]
            blurred_channel = sp.signal.convolve2d(channel, gaussian_kernel, boundary="symm", mode="same")
            channels.append(blurred_channel)
        image = np.dstack(channels)
        g_stack.append(image)
    return g_stack

def laplacian_stack(image):
    g_stack = gaussian_stack(image)
    lp_stack = []
    for i in range(len(g_stack)-1):
        lp_level = g_stack[i] - g_stack[i+1]
        lp_stack.append(lp_level)
    lp_stack.append(g_stack[-1])  
    return lp_stack

def blend_images(imageA, imageB, mask):
    lp_A = laplacian_stack(imageA)
    lp_B = laplacian_stack(imageB)
    g_stack_mask = gaussian_stack(mask)
    collapse = np.zeros(lp_A[0].shape)
    for i in range(len(g_stack_mask)):
        level_A = lp_A[i]
        level_B = lp_B[i]
        level_mask = g_stack_mask[i]
        new_image = level_mask*level_A + (1-level_mask)*level_B
        collapse = collapse + new_image
    return collapse

In [ ]:
def gaussian_stitch(im1,im2):
    overlap_mask = np.any((im1 != 0), axis = 2) & np.any(im2 != 0, axis = 2)
    overlap_cols = np.where(np.any(overlap_mask,axis=0))[0]
    gradient = np.linspace(1,0,num = overlap_cols.shape[0])
    overlap_gradient = np.zeros(im1.shape)
    
    # Create overlap gradient mask
    for col,val in zip(overlap_cols,gradient):
        overlap_gradient[overlap_mask[:,col],col,:] = val
   
    # Create im1 mask
    im1_mask = np.any(im1 != 0, axis = 2)
    im1_mask_gradient = np.zeros(im1.shape)
    im1_mask_gradient[im1_mask,:] = 1
    im1_mask_gradient[overlap_mask] = overlap_gradient[overlap_mask]

    # Create im2 mask
    im2_mask = np.any(im2 != 0, axis = 2)
    im2_mask_gradient = np.zeros(im2.shape)
    im2_mask_gradient[im2_mask,:] = 1
    im2_mask_gradient[overlap_mask] = 1-overlap_gradient[overlap_mask]

    # Stitch images
    stitched_im = im1*im1_mask_gradient + im2*im2_mask_gradient

    return np.clip(stitched_im,0,1)
    

def blend_mosaic(images, images_pts):
    # Warp each image onto the first one
    ref_im = images[0]
    ref_im_pts = images_pts[0]
    warped_images = [ref_im]
    for i in range(1,len(images)):
        im = images[i]
        im_pts = images_pts[i]
        H_im = computeH(im_pts, ref_im_pts)
        warped_im = warpImage(im, H_im)
        warped_images.append(warped_im)

    # Project each image onto larger mosaic plane
    size_x = max([im.shape[0] for im in warped_images])
    size_y = max([im.shape[1] for im in warped_images])
    mosaic_ims = []
    for im in warped_images:
        projected_im = np.zeros((size_x,size_y,3))
        projected_im[:im.shape[0],:im.shape[1],:] = im
        mosaic_ims.append(projected_im)

    # Stitch images
    final_mosaic = mosaic_ims[0]
    for i in range(1,len(mosaic_ims)):
        final_mosaic = gaussian_stitch(final_mosaic, mosaic_ims[i])
        
    return final_mosaic


    

In [ ]:
# Load images of park
park_left = sk.img_as_float(skio.imread("park_left.jpg"))
park_middle = sk.img_as_float(skio.imread("park_middle.jpg"))
park_right = sk.img_as_float(skio.imread("park_right.jpg"))
fig,axes = plt.subplots(1,3)
axes[0].imshow(park_left)
axes[1].imshow(park_middle)
axes[2].imshow(park_right)

In [ ]:
# Mosaic of park
park_left_pts = np.array([[ 310,  945],
 [ 432,  958],
 [ 561, 1032],
 [ 601, 1087],
 [ 532, 1126],
 [ 467, 1135],
 [ 412, 1185]])

park_middle_pts = np.array([[299, 475],
 [417, 485],
 [548, 556],
 [583, 598],
 [523, 633],
 [462, 642],
 [415, 680]])

park_right_pts = np.array([[295, 22],
 [391, 36],
 [545, 125],
 [575, 177],
 [512, 222],
 [444, 235],
 [393, 280]])

In [ ]:
park_ims = [park_left,park_middle,park_right]
park_pts = [park_left_pts,park_middle_pts,park_right_pts]
park_mosaic = blend_mosaic(park_ims,park_pts)

In [ ]:
plt.imshow(park_mosaic)

In [ ]:
# Load DL images
DL_left = sk.img_as_float(skio.imread("images/DL_left.jpg"))
DL_right = sk.img_as_float(skio.imread("images/DL_right.jpg"))
fig,axes = plt.subplots(1,2)
axes[0].imshow(DL_left)
axes[1].imshow(DL_right)

In [ ]:
DL_left_pts = np.array([[ 249,  561],
 [ 506,  577],
 [ 123,  881],
 [ 115,  929],
 [ 168,  881],
 [ 156,  931],
 [ 388, 1062],
 [ 349, 1066],
 [ 744, 1038],
 [ 452, 1142],
 [ 584, 1130],
 [ 697, 1110]])


DL_right_pts = np.array([[244,  33],
 [544,  76],
 [136, 384],
 [131, 429],
 [171, 384],
 [171, 436],
 [392, 565],
 [355, 563],
 [707, 575],
 [442, 627],
 [563, 633],
 [658, 621]])

In [ ]:
plt.imshow(DL_left)
plt.plot(DL_left_pts[:,1],DL_left_pts[:,0],'ro')

In [ ]:
plt.imshow(DL_right)
plt.plot(DL_right_pts[:,1],DL_right_pts[:,0],'ro')

In [ ]:
DL_mosaic = blend_mosaic([DL_left, DL_right], [DL_left_pts, DL_right_pts])

In [ ]:
plt.imshow(DL_mosaic)
skio.imsave("DL_mosaic.jpg", sk.img_as_ubyte(DL_mosaic))

In [ ]:
# Load Campanile Images
campanile_left = sk.img_as_float(skio.imread("images/campanile_left.jpg"))
campanile_middle = sk.img_as_float(skio.imread("images/campanile_middle.jpg"))
fig,axes = plt.subplots(1,2)
axes[0].imshow(campanile_left)
axes[1].imshow(campanile_middle)

In [ ]:
campanile_left_pts = np.array([[  53,  976],
 [ 138,  943],
 [ 179,  934],
 [ 248,  938],
 [ 272,  964],
 [ 278, 1009],
 [ 245, 1022],
 [ 182, 1023],
 [ 139, 1012],
 [ 130,  984],
 [ 526, 1054],
 [ 655,  965],
 [ 657, 1032],
 [ 657, 1098],
 [ 544,  938],
 [ 545,  978]])


campanile_middle_pts = np.array([[ 92, 590],
 [166, 555],
 [210, 545],
 [275, 544],
 [299, 571],
 [307, 613],
 [278, 629],
 [212, 629],
 [171, 623],
 [163, 593],
 [549, 652],
 [676, 562],
 [677, 625],
 [679, 694],
 [568, 541],
 [565, 579]])

In [ ]:
plt.imshow(campanile_left)
plt.plot(campanile_left_pts[:,1],campanile_left_pts[:,0],'ro')
# for i in range(campanile_left_pts.shape[0]):
#     plt.annotate(i,(campanile_left_pts[:,1][i],campanile_left_pts[:,0][i]))

In [ ]:
plt.imshow(campanile_middle)
plt.plot(campanile_middle_pts[:,1],campanile_middle_pts[:,0],'ro')
# for i in range(campanile_middle_pts.shape[0]):
#     plt.annotate(i,(campanile_middle_pts[:,1][i],campanile_middle_pts[:,0][i]))

In [ ]:
campanile_mosaic = blend_mosaic([campanile_left, campanile_middle],[campanile_left_pts, campanile_middle_pts])
plt.imshow(campanile_mosaic)

In [ ]:
skio.imsave("campanile_mosaic.jpg", sk.img_as_ubyte(campanile_mosaic))

In [ ]:
# Do mosaic of Sproul images
sproul_left = sk.img_as_float(skio.imread("images/sproul_left.jpg"))
sproul_right = sk.img_as_float(skio.imread("images/sproul_right.jpg"))

In [ ]:
plt.imshow(sproul_left)

In [ ]:
plt.imshow(sproul_right)

In [ ]:
sproul_left_pts = np.array([[ 195,  681],
 [ 288,  682],
 [ 398,  656],
 [ 521,  676],
 [ 293,  763],
 [ 390,  708],
 [ 515,  763],
 [ 639,  832],
 [ 642,  914],
 [ 641, 1046],
 [ 545,  943],
 [ 634,  943],
 [ 337, 1067],
 [  84,  779],
 [ 396,  860],
 [ 502, 1080]])

sproul_right_pts = np.array([[146, 127],
 [255, 131],
 [375, 100],
 [516, 129],
 [266, 230],
 [372, 166],
 [506, 236],
 [626, 309],
 [619, 393],
 [602, 510],
 [524, 420],
 [608, 418],
 [338, 523],
 [ 47, 246],
 [381, 339],
 [475, 535]])

In [ ]:
plt.imshow(sproul_left)
plt.plot(sproul_left_pts[:,1],sproul_left_pts[:,0],'ro')
# for i in range(sproul_left_pts.shape[0]):
#     plt.annotate(i,(sproul_left_pts[:,1][i],sproul_left_pts[:,0][i]))

In [ ]:
plt.imshow(sproul_right)
plt.plot(sproul_right_pts[:,1],sproul_right_pts[:,0],'ro')
# for i in range(sproul_right_pts.shape[0]):
#     plt.annotate(i,(sproul_right_pts[:,1][i],sproul_right_pts[:,0][i]))

In [ ]:
sproul_mosaic = blend_mosaic([sproul_left, sproul_right], [sproul_left_pts, sproul_right_pts])
plt.imshow(sproul_mosaic)

In [ ]:
skio.imsave("sproul_mosaic.jpg", sk.img_as_ubyte(sproul_mosaic))

# Feature Matching for Autostitching - Part B

In [ ]:
import os
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import skimage as sk
import skimage.io as skio
import cv2
from skimage.feature import corner_harris, peak_local_max

In [ ]:
def get_harris_corners(im, edge_discard=20):
    """
    This function takes a b&w image and an optional amount to discard
    on the edge (default is 5 pixels), and finds all harris corners
    in the image. Harris corners near the edge are discarded and the
    coordinates of the remaining corners are returned. A 2d array (h)
    containing the h value of every pixel is also returned.

    h is the same shape as the original image, im.
    coords is 2 x n (ys, xs).
    """

    assert edge_discard >= 20

    # find harris corners
    h = corner_harris(im, method='eps', sigma=5)
    coords = peak_local_max(h, min_distance=1, threshold_abs = 10/256)

    # discard points on edge
    edge = edge_discard  # pixels
    mask = (coords[:, 0] > edge) & \
           (coords[:, 0] < im.shape[0] - edge) & \
           (coords[:, 1] > edge) & \
           (coords[:, 1] < im.shape[1] - edge)
    coords = coords[mask].T
    return h, coords


def dist2(x, c):
    """
    dist2  Calculates squared distance between two sets of points.

    Description
    D = DIST2(X, C) takes two matrices of vectors and calculates the
    squared Euclidean distance between them.  Both matrices must be of
    the same column dimension.  If X has M rows and N columns, and C has
    L rows and N columns, then the result has M rows and L columns.  The
    I, Jth entry is the  squared distance from the Ith row of X to the
    Jth row of C.

    Adapted from code by Christopher M Bishop and Ian T Nabney.
    """
    
    ndata, dimx = x.shape
    ncenters, dimc = c.shape
    assert(dimx == dimc, 'Data dimension does not match dimension of centers')

    return (np.ones((ncenters, 1)) * np.sum((x**2).T, axis=0)).T + \
            np.ones((   ndata, 1)) * np.sum((c**2).T, axis=0)    - \
            2 * np.inner(x, c)


In [ ]:
campanile_middle = sk.img_as_float(skio.imread("images/campanile_left.jpg"))
plt.imshow(campanile_middle)

In [ ]:
H_campanile, coords_campanile = get_harris_corners(campanile_middle[:,:,0])

In [ ]:
plt.imshow(campanile_middle)
plt.plot(coords_campanile[1,:], coords_campanile[0,:],'ro')

In [ ]:
def ANMS(H, coords):
    c_robust = 0.9
    num_pts = 100
    pts_r_i = []
    for x,y in coords.T:
        # Compute r_i
        window = H[x-3:x+3,y-3:y+3]
        threshold_pts = np.argwhere(window < c_robust*H[x,y])
        #print(dist2(np.array([[0,0]]),threshold_pts).shape)
        r_i = np.min(dist2(np.array([[0,0]]),threshold_pts), initial=float('inf'))
        pts_r_i.append([x,y,r_i])
        
    # Find num_pts points with highest r_i
    sorted_pts_r_i = np.array(sorted(pts_r_i, key=lambda x:x[2]))
    anms_pts = sorted_pts_r_i[:num_pts,:2]
    return anms_pts.astype(int)

In [ ]:
campanile_anms_pts = ANMS(H_campanile, coords_campanile)

In [ ]:
plt.imshow(campanile_middle)
plt.plot(campanile_anms_pts[:,1], campanile_anms_pts[:,0],'ro')

In [ ]:
def extract_feature_patches(im, ft_pts):
    patches = []
    for x,y in ft_pts:
        ft_patch =  im[x-20:x+20,y-20:y+20,:]
        # Normalize and resize patch
        ft_patch = (ft_patch - np.mean(ft_patch)) / np.std(ft_patch)
        ft_patch = sk.transform.resize(ft_patch, (8,8,3))
        patches.append([x,y, ft_patch])  
    return patches   

def matching_outlier_rejection(im1_patches, im2_patches):
    threshold = 0.6
    keep_im1_pts = []
    keep_im2_pts = []
    for i in range(len(im1_patches)):
        one_nn = float('inf')
        second_nn = float('inf')
        best_j = -1
        x1,y1,im1_patch = im1_patches[i]
        for j in range(len(im2_patches)):
            x2,y2,im2_patch = im2_patches[j]
            ssd_val = np.sum(np.square(im1_patch - im2_patch))
            if ssd_val < one_nn:
                one_nn = ssd_val
                best_j = j
            elif ssd_val < second_nn:
                second_nn = ssd_val 
                
        # Keep if point is not an outlier by Lowe's Trick        
        if one_nn/second_nn < threshold:
            best_x2, best_y2, _ = im2_patches.pop(best_j)
            keep_im1_pts.append([x1,y1])
            keep_im2_pts.append([best_x2,best_y2])
            
    return np.array(keep_im1_pts), np.array(keep_im2_pts)                    

In [ ]:
def RANSAC(im1_pts, im2_pts):
    num_iterations = 1000
    epsilon = 5
    max_inliers = []
    for _ in range(num_iterations):
        # Pick random samples of 4 points from each image
        pts_rand = np.random.choice(im1_pts.shape[0], size=4)
        im1_sample = im1_pts[pts_rand,:]
        im2_sample = im2_pts[pts_rand,:]
        
        # Compute homography
        H = computeH(im1_sample, im2_sample)

        # Count num inliers
        im1_mapped_pts = H @ np.vstack([im1_pts.T, np.ones(im1_pts.shape[0])])
        im1_mapped_pts = im1_mapped_pts.T[:,:2] / im1_mapped_pts.T[:,2].reshape(-1,1)
        inliers = []
        for i in range(im2_pts.shape[0]):
            ssd_val = np.sum(np.square(im1_mapped_pts[i]-im2_pts[i]))
            if ssd_val < epsilon:
                inliers.append(i)
        if len(inliers) > len(max_inliers):
            max_inliers = inliers

    # Compute final homography on larget set of inliers found
    im1_inliers = im1_pts[max_inliers,:]
    im2_inliers = im2_pts[max_inliers,:]
    final_H = computeH(im1_inliers, im2_inliers)
    return final_H

In [ ]:
def autoH(im1, im2):
    H_im1, coords_im1 = get_harris_corners(im1[:,:,0])
    H_im2, coords_im2 = get_harris_corners(im2[:,:,0])
    im1_ft_pts = ANMS(H_im1, coords_im1)
    im2_ft_pts = ANMS(H_im2, coords_im2)
    im1_patches = extract_feature_patches(im1,im1_ft_pts)
    im2_patches = extract_feature_patches(im2,im2_ft_pts)
    im1_pts, im2_pts = matching_outlier_rejection(im1_patches, im2_patches)
    H = RANSAC(im1_pts, im2_pts)
    return im1_patches, im2_patches, coords_im1, coords_im2, im1_ft_pts, im2_ft_pts, im1_pts, im2_pts, H

def autoStitch(im1, im2, H):
    warped_images = [im1, warpImage(im2,H)]
    
    # Project each image onto larger mosaic plane
    size_x = max([im.shape[0] for im in warped_images])
    size_y = max([im.shape[1] for im in warped_images])
    mosaic_ims = []
    for im in warped_images:
        projected_im = np.zeros((size_x,size_y,3))
        projected_im[:im.shape[0],:im.shape[1],:] = im
        mosaic_ims.append(projected_im)

    # Stitch images
    final_mosaic = mosaic_ims[0]
    for i in range(1,len(mosaic_ims)):
        final_mosaic = gaussian_stitch(final_mosaic, mosaic_ims[i])
        
    return final_mosaic

In [ ]:
# Auto-stitch DL mosaic
DL_left = sk.img_as_float(skio.imread("images/DL_left.jpg"))
DL_right = sk.img_as_float(skio.imread("images/DL_right.jpg"))
DL_auto = autoH(DL_right, DL_left)
DL_mosaic = autoStitch(DL_left, DL_right, DL_auto[-1])

In [ ]:
plt.imshow(DL_mosaic)
skio.imsave("DL_auto_mosaic.jpg", sk.img_as_ubyte(DL_mosaic))

In [ ]:
# Auto-stitch campanile mosaic
campanile_left = sk.img_as_float(skio.imread("images/campanile_left.jpg"))
campanile_middle = sk.img_as_float(skio.imread("images/campanile_middle.jpg"))
campanile_auto = autoH(campanile_middle, campanile_left)
campanile_mosaic = autoStitch(campanile_left, campanile_middle, campanile_auto[-1])

In [ ]:
# Campanile Left Matched Pts
plt.figure(figsize=(10,10))
plt.imshow(campanile_left)
plt.plot(campanile_auto[-2][:,1],campanile_auto[-2][:,0],'ro')

In [ ]:
# Campanile Middle Matched Pts
plt.figure(figsize=(10,10))
plt.imshow(campanile_middle)
plt.plot(campanile_auto[-3][:,1],campanile_auto[-3][:,0],'ro')

In [ ]:
# Campanile Left ANMS Pts
plt.figure(figsize=(10,10))
plt.imshow(campanile_left)
plt.plot(campanile_auto[-4][:,1],campanile_auto[-4][:,0],'ro')

In [ ]:
# Campanile Middle ANMS Pts
plt.figure(figsize=(10,10))
plt.imshow(campanile_middle)
plt.plot(campanile_auto[-5][:,1],campanile_auto[-5][:,0],'ro')

In [ ]:
# Campanile Left Harris Pts
plt.figure(figsize=(10,10))
plt.imshow(campanile_left)
plt.plot(campanile_auto[-6].T[:,1],campanile_auto[-6].T[:,0],'ro')

In [ ]:
# Campanile Middle Harris Pts
plt.figure(figsize=(10,10))
plt.imshow(campanile_middle)
plt.plot(campanile_auto[-7].T[:,1],campanile_auto[-7].T[:,0],'ro')

In [ ]:
plt.imshow(campanile_mosaic)
skio.imsave("campanile_auto_mosaic.jpg", sk.img_as_ubyte(campanile_mosaic))

In [ ]:
len(campanile_auto[1])

In [ ]:
# Plot feature descriptors for left
fig = plt.figure(figsize=(20,20))
for i in range(len(campanile_auto[1])):
    ft_d = campanile_auto[1][i][-1]
    ft_d = (ft_d - np.min(ft_d)) / (np.max(ft_d)-np.min(ft_d))
    fig.add_subplot(6,10,i+1)
    plt.axis('off')
    plt.grid(False)
    plt.subplots_adjust(top=0.5, bottom=0.1, left=0.1, right=0.9)
    plt.imshow(ft_d)

In [ ]:
# Plot feature descriptors for middle
fig = plt.figure(figsize=(30,30))
for i in range(len(campanile_auto[0])):
    ft_d = campanile_auto[0][i][-1]
    ft_d = (ft_d - np.min(ft_d)) / (np.max(ft_d)-np.min(ft_d))
    fig.add_subplot(10,10,i+1)
    plt.axis('off')
    plt.grid(False)
    plt.subplots_adjust(top=0.5, bottom=0.1, left=0.1, right=0.5)
    plt.imshow(ft_d)

In [ ]:
# Auto-stitch Sproul mosaic
sproul_left = sk.img_as_float(skio.imread("images/sproul_left.jpg"))
sproul_right = sk.img_as_float(skio.imread("images/sproul_right.jpg"))
sproul_auto = autoH(sproul_right, sproul_left)
sproul_mosaic = autoStitch(sproul_left, sproul_right, sproul_auto[-1])

In [ ]:
plt.imshow(sproul_mosaic)
skio.imsave("sproul_auto_mosaic.jpg", sk.img_as_ubyte(sproul_mosaic))